<a href="https://colab.research.google.com/github/ramanathanlab/genslm/blob/main/examples/generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# NOTE: You may need to run this twice due to a pip dependency conflict
!pip install git+https://github.com/ramanathanlab/genslm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/ramanathanlab/genslm to /tmp/pip-req-build-3222q863
  Running command git clone -q https://github.com/ramanathanlab/genslm /tmp/pip-req-build-3222q863
  Cloning https://github.com/maxzvyagin/transformers to /tmp/pip-install-k_9gz20w/transformers_f804a482b2c84233b81b363bb7ed240d
  Running command git clone -q https://github.com/maxzvyagin/transformers /tmp/pip-install-k_9gz20w/transformers_f804a482b2c84233b81b363bb7ed240d
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [3]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [11]:
!ls gdrive/MyDrive/patric_25m_epoch01-val_loss_0.57_bias_removed.pt
# This currently requires you to download the 25M model weights from Globus

gdrive/MyDrive/patric_25m_epoch01-val_loss_0.57_bias_removed.pt


In [1]:
import torch
from genslm import GenSLM
from Bio.Seq import Seq

In [2]:
# Load model
#model = GenSLM("genslm_25M_patric", model_cache_dir="/content/gdrive/MyDrive")
model = GenSLM("genslm_2.5B_patric", model_cache_dir="/lus/eagle/projects/FoundEpidem/xlian/genslm_models/2.5B")
model.eval()

# Select GPU device if it is available, else use CPU
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

2025-03-31 21:22:08.604577: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-31 21:22:44.725856: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


GenSLM(
  (model): GPTNeoXForCausalLM(
    (gpt_neox): GPTNeoXModel(
      (embed_in): Embedding(69, 3840)
      (emb_dropout): Dropout(p=0.0, inplace=False)
      (layers): ModuleList(
        (0-27): 28 x GPTNeoXLayer(
          (input_layernorm): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
          (post_attention_layernorm): LayerNorm((3840,), eps=1e-05, elementwise_affine=True)
          (post_attention_dropout): Dropout(p=0.0, inplace=False)
          (post_mlp_dropout): Dropout(p=0.0, inplace=False)
          (attention): GPTNeoXAttention(
            (query_key_value): Linear(in_features=3840, out_features=11520, bias=True)
            (dense): Linear(in_features=3840, out_features=3840, bias=True)
          )
          (mlp): GPTNeoXMLP(
            (dense_h_to_4h): Linear(in_features=3840, out_features=4096, bias=True)
            (dense_4h_to_h): Linear(in_features=4096, out_features=3840, bias=True)
            (act): FastGELUActivation()
          )
        )
 

In [28]:
vanA = 'atgattgaggtaatcatttcggcgatgcgcttggttgctcaggacatcattagccttgagtttgtccgggctgacggtggcttgcttccgcctgtcgaggccggcgcccacgtcgatgtgcatcttcctggcggcctgattcggcagtactcgctctggaatcaaccaggggcgcagagccattactgcatcggtgttctgaaggacccggcgtctcgtggtggttcgaaggcggtgcacgagaatcttcgcgtcgggatgcgcgtgcaaattagcgagccgaggaacctattcccattggaagagggggtggagcggagtctgctgttcgcgggcgggattggcattacgccgattctgtgtatggctcaagaattagcagcacgcgagcaagatttcgagttgcattattgcgcgcgttcgaccgaccgagcggcgttcgttgaatggcttaaggtttgcgactttgctgatcacgtacgtttccactttgacaatggcccggatcagcaaaaactgaatgccgcagcgctgctagcggccgaggccgaaggtacccacctttatgtctgtgggcccggcgggttcatggggcatgtgcttgataccgcgaaggagcagggctgggctgacaatcgactgcatcgagagtatttcgccgcggcgccgaatgtgagtgctgacgatggcagtttcgaggtgcggattcacagcaccggacaagtgcttcaggtccccgcggatcaaacggtctcccaggtgctcgatgcggccggaattatcgttcccgtttcttgtgagcagggcatctgcggtacttgcatcactcgggtggtagacggagagcctgatcatcgtgacttcttcctcacggatgcggagaaggcaaagaacgaccagttcaccccctgttgctcgcgagccaagagcgcctgtttggtcttggatctctaa'
vanA = vanA.upper()

if len(vanA) % 3 != 0:
    raise ValueError(f"DNA sequence length ({len(vanA)}) is not divisible by 3")
    
codons = [vanA[i:i+3] for i in range(0, len(vanA), 3)]

In [29]:
# Prompt the language model with a start codon
prom = ' '.join(codons[:100])
prompt = model.tokenizer.encode(' '.join(codons[:100]), return_tensors="pt").to(device)

tokens = model.model.generate(
    prompt,
    max_length=275,  # Increase this to generate longer sequences
    min_length=225,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=2,  # Change the number of sequences to generate
    remove_invalid_values=True,
    use_cache=True,
    pad_token_id=model.tokenizer.encode("[PAD]")[0],
    temperature=1.0,
)

sequences = model.tokenizer.batch_decode(tokens, skip_special_tokens=True)
print('generated')

generated


In [30]:
def translate_codon_sequences(sequences):
    proteins = []
    for seq_str in sequences:
        # Remove spaces and create a Biopython Seq object
        dna_seq = Seq(seq_str.replace(" ", "").upper())
        protein = dna_seq.translate(to_stop=True)
        proteins.append(str(protein))
    return proteins

proteins = translate_codon_sequences(sequences)
for i in proteins:
    print(i)

MIEVIISAMRLVAQDIISLEFVRADGGLLPPVEAGAHVDVHLPGGLIRQYSLWNQPGAQSHYCIGVLKDPASRGGSKAVHENLRVGMRVQISEPRNLFPLIEAPGRTILVAGGIGITPLMAMAWRLQAEGLSFASLYCGRSRGSMAFLEALSSGARSRFVTRADEAGVLGDEEFLLAVLKTPPTPYLYVCGPAAMLQAVISAAQEAGVTAERIHSEYFAAPTPVDTGSFEIQVKRSGKVVVIDGADSPLVQLGGDVLEVLARRADQPFETCCEGP
MIEVIISAMRLVAQDIISLEFVRADGGLLPPVEAGAHVDVHLPGGLIRQYSLWNQPGAQSHYCIGVLKDPASRGGSKAVHENLRVGMRVQISEPRNLFPLARRAHRTLLFGGGIGVTPLLAMAYELKQSGEDFALHYCVRSRENAAFIDDIKTASHAKDVFFHFDDEAAGRLDLPGALLNRQPDTHLYVCGPGGFMEYVLNTARAAGWPEEHLHREYFGAAPADGSTTDKAFVVRVHTSGKLQVSPDDDVAQVILEAGISVPVSCEQGVCGTCLT


In [33]:
print(Seq(''.join(codons)).translate())

MIEVIISAMRLVAQDIISLEFVRADGGLLPPVEAGAHVDVHLPGGLIRQYSLWNQPGAQSHYCIGVLKDPASRGGSKAVHENLRVGMRVQISEPRNLFPLEEGVERSLLFAGGIGITPILCMAQELAAREQDFELHYCARSTDRAAFVEWLKVCDFADHVRFHFDNGPDQQKLNAAALLAAEAEGTHLYVCGPGGFMGHVLDTAKEQGWADNRLHREYFAAAPNVSADDGSFEVRIHSTGQVLQVPADQTVSQVLDAAGIIVPVSCEQGICGTCITRVVDGEPDHRDFFLTDAEKAKNDQFTPCCSRAKSACLVLDL*


In [25]:
prompt = model.tokenizer.encode('AAA AAA', return_tensors="pt").to(device)
prompt

tensor([[16, 16]], device='cuda:0')